In [ ]:
import mne
from datetime import datetime

from bis_lib import *
from bis_lib_feature import *
from bis_lib_noise import get_noise_idx

In [ ]:
noise_idx_all = get_noise_idx()

In [ ]:
print(noise_idx_all.keys())

In [ ]:
data_info = {}
draw_c_nc = {}
xy_learning = {}
feature_cal = True
fs = 1000
seg_length = 5
exp_dates = ['0428', '0516', '0517', '0518', '0519', '0523', '0524', '0525', '0526']

# for exp_date in exp_dates:
for exp_date in ['0526']:
    path_root = '/Users/zhangchao/Downloads/data_tmp/{}'.format(exp_date)
    exp = get_exp(path_root, exp_date)
    eeg_start_time, eeg_end_time = get_eeg_time(path_root, exp)
    csv_path = '../csv_files/{}'.format(exp_date)
    # for i in range(len(exp)):
    for i in range(2,3):
        eeg = csv2eeg(path_root, exp[i])
        print('eeg.shape', eeg.shape)
        print('duration(s):', eeg.shape[1]/fs)
        data_info['exp{}_duration'.format(i)] = eeg.shape[1]/fs
        tmp = datetime.strptime(eeg_end_time[i], '%Y-%m-%d %H:%M:%S') - datetime.strptime(eeg_start_time[i], '%Y-%m-%d %H:%M:%S')
        data_info['exp{}_pack_loss'.format(i)] = tmp.seconds - eeg.shape[1]/fs

        ch_num = eeg.shape[0]
        # for mne object
        ch_names = ['EEG{}'.format(j) for j in range(ch_num)]
        ch_types = ['eeg' for i in range(ch_num)]
        info = mne.create_info(ch_names, ch_types=ch_types, sfreq=fs)
        print(info)

        data_info['exp{}_chnum'.format(i)] = eeg.shape[0]
        data_info['exp{}_filename'.format(i)] = exp[i]

        my_raw = mne.io.RawArray(eeg, info)
        my_raw.filter(l_freq=0.5, h_freq=30)
        # draw_eeg_psd(my_raw, ch_names, exp[i])

        if exp_date in ['0523', '0524', '0525', '0526']:
            xls_info = process_xls_new(csv_path, exp_id=i, eeg_start_time=eeg_start_time, eeg_end_time=eeg_end_time)
        else:
            xls_info = process_xls(csv_path, exp_id=i, eeg_start_time=eeg_start_time, eeg_end_time=eeg_end_time)
            
        eeg_data = eeg_segment(my_raw[:][0], fs, seg_length=seg_length, eeg_start_time=eeg_start_time[i])
        eeg_data_tmp = eeg_segment(my_raw[:][0], fs, seg_length=15, eeg_start_time=eeg_start_time[i])

        # eeg_feature_dict = {}
        if feature_cal:
            pe = get_pe_clean(eeg_data, seg_length, noise_idx_all['{}_{}'.format(exp_date, i+1)])
            lzc = get_lzc_clean(eeg_data, seg_length, noise_idx_all['{}_{}'.format(exp_date, i+1)])
            svd = get_svd_clean(eeg_data_tmp, 15, noise_idx_all['{}_{}'.format(exp_date, i+1)], fs)
            # pe = get_pe(eeg_data)
            # se = get_se(eeg_data)
            # lzc = get_lzc(eeg_data)
            # svd = get_conn_svd(eeg_data, fs)
            # bism_pe = get_bism_pe(eeg_data, seg_length)
            
        else:
            pe = np.load('../npy_files/feature_clean/PE_{}_{}.npy'.format(exp_date, exp[i]))
            # se = np.load('../npy_files/feature_clean/SE_{}_{}.npy'.format(exp_date, exp[i]))
            lzc = np.load('../npy_files/feature_clean/LZC_{}_{}.npy'.format(exp_date, exp[i]))
            svd = np.load('../npy_files/feature_clean/SVD_{}_{}.npy'.format(exp_date, exp[i]))

        # eeg_feature_dict['PE'] = np.mean(pe, axis=1)
        # # eeg_feature_dict['SE'] = se
        # eeg_feature_dict['LZC'] = np.mean(lzc, axis=1)
        # eeg_feature_dict['SVD_0'] = svd[0,:]
        # eeg_feature_dict['SVD_1'] = svd[1,:]
        # eeg_feature_dict['SVD_2'] = svd[2,:]
        # eeg_feature_dict['SVD_3'] = svd[3,:]
        
        # if feature_cal:
        #     np.save('../npy_files/feature_clean/PE_{}_{}.npy'.format(exp_date, exp[i]), pe)
        #     # np.save('../npy_files/SE_{}_{}.npy'.format(exp_date, exp[i]), se)
        #     np.save('../npy_files/feature_clean/LZC_{}_{}.npy'.format(exp_date, exp[i]), lzc)
        #     np.save('../npy_files/feature_clean/SVD_{}_{}.npy'.format(exp_date, exp[i]), svd)

        # draw_results(exp_date, exp[i], xls_info, eeg_feature_dict)